In [9]:
#!pip install youtube_transcript_api
#!pip install youtube-dl
#!pip install openai

# Podcast Summarization

This code takes a link from a video and creates a summarization

## Transcript extraction
This code extracts the trasncript it is here as a test. Not used in the final code

In [7]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter

def get_youtube_transcript(video_id):
    try:
        # Fetch the transcript
        transcript = YouTubeTranscriptApi.get_transcript(video_id)

        # Format the transcript into plain text
        formatter = TextFormatter()
        transcript_text = formatter.format_transcript(transcript)

        return transcript_text

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# Example usage
video_url = 'https://www.youtube.com/watch?v=xA5B6quoahY'  # Replace with your YouTube video URL
video_id = video_url.split('v=')[1]  # Extract the video ID from the URL

transcript = get_youtube_transcript(video_id)

if transcript:
    print("Transcript extracted successfully:")
    print(transcript)
else:
    print("No transcript found or an error occurred.")


Transcript extracted successfully:
all right everybody Welcome to the
number one podcast in the world I am
your boy J Cal the world's greatest
moderator executive producer for life
with me again on the all-in podcast is
the Rainman David Sachs yeah looking
healthy looking good looking trim the
hair is great how you feeling brother
good good haircut good haircut looks
good now do you go to the barber The
Stylist they come to you everybody wants
to know they come to me yeah course they
do of course
and then with us of course your sulan of
science he loves those vegetables his
name is David Friedberg uh from Burkle
to the Bay our boy how you doing there
brother are you living the lake
lifestyle you have a nice restful August
loving loving the outdoors good for you
you look so healthy speaking of looking
healthy celebrated birthday with Jason
 yesterday happy 39th Jason
happy 39th Jason we did a long
mountain bike ride with our other buddy
Z and had a little family Barbecue last
night ever

This code attempts to identify the different speakers. Not used in the final code

In [8]:
from youtube_transcript_api import YouTubeTranscriptApi
import re

def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
    except Exception as e:
        print(f"Error: {e}")
        return None
    return transcript

def identify_speakers(transcript):
    """
    This is a simple heuristic-based approach to identify different speakers.
    In a real-world scenario, speaker diarization and advanced NLP would be needed.
    """
    identified_transcript = []
    current_speaker = "Speaker 1"
    previous_time = 0

    for entry in transcript:
        text = entry['text']
        start_time = entry['start']

        # Heuristic: If there's a significant gap between the start times, assume a speaker change
        if start_time - previous_time > 5:  # 5 seconds gap heuristic
            current_speaker = "Speaker 2" if current_speaker == "Speaker 1" else "Speaker 1"

        identified_transcript.append(f"{current_speaker}: {text}")
        previous_time = start_time

    return identified_transcript

def analyze_tone(text):
    # Placeholder for tone analysis - here you'd use a tone analysis API
    # For example, using IBM Watson Tone Analyzer (not implemented)
    # tone_analysis = tone_analyzer.tone({'text': text}, content_type='application/json').get_result()
    # return tone_analysis
    return "Neutral"  # Simple placeholder

def main(video_url):
    # Extract video ID from URL
    video_id = re.search(r'v=([a-zA-Z0-9_-]+)', video_url).group(1)
    
    # Get the transcript
    transcript = get_transcript(video_id)
    if transcript is None:
        return
    
    # Identify speakers
    identified_transcript = identify_speakers(transcript)

    # Analyze tone (not fully implemented in this example)
    transcript_with_tone = []
    for line in identified_transcript:
        tone = analyze_tone(line)
        transcript_with_tone.append(f"{line} [{tone} tone]")

    # Output the final transcript
    final_transcript = "\n".join(transcript_with_tone)
    print(final_transcript)

# Example usage
video_url = "https://www.youtube.com/watch?v=xA5B6quoahY"  # Replace with your YouTube URL
main(video_url)


Speaker 1: all right everybody Welcome to the [Neutral tone]
Speaker 1: number one podcast in the world I am [Neutral tone]
Speaker 1: your boy J Cal the world's greatest [Neutral tone]
Speaker 1: moderator executive producer for life [Neutral tone]
Speaker 1: with me again on the all-in podcast is [Neutral tone]
Speaker 1: the Rainman David Sachs yeah looking [Neutral tone]
Speaker 1: healthy looking good looking trim the [Neutral tone]
Speaker 1: hair is great how you feeling brother [Neutral tone]
Speaker 1: good good haircut good haircut looks [Neutral tone]
Speaker 1: good now do you go to the barber The [Neutral tone]
Speaker 1: Stylist they come to you everybody wants [Neutral tone]
Speaker 1: to know they come to me yeah course they [Neutral tone]
Speaker 1: do of course [Neutral tone]
Speaker 1: and then with us of course your sulan of [Neutral tone]
Speaker 1: science he loves those vegetables his [Neutral tone]
Speaker 1: name is David Friedberg uh from Burkle [Neutral tone]

In [67]:
#!pip install openai

In [76]:
def read_api_key(file_path):
    """
    Reads the API key from a local file.

    Args:
        file_path (str): The path to the file containing the API key.

    Returns:
        str: The API key.
    """
    try:
        with open(file_path, 'r') as file:
            api_key = file.read().strip()
        return api_key
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Usage
api_key = read_api_key('C:/Users/caror/OneDrive/Documents/Keys/chatgpt.txt')
if api_key:
    print("API key read successfully!")
else:
    print("Failed to read API key.")


API key read successfully!


In [81]:
import openai
from youtube_transcript_api import YouTubeTranscriptApi
import re

# Set your OpenAI API key here
openai.api_key = api_key

def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
    except Exception as e:
        print(f"Error: {e}")
        return None
    return transcript

def clean_transcript(transcript):
    """Remove any jokes or irrelevant content heuristically."""
    cleaned_transcript = []
    for entry in transcript:
        text = entry['text']
        
        # Simple heuristic to exclude jokes or shenanigans
        if any(keyword in text.lower() for keyword in ["haha", "joke", "funny", "laugh"]):
            continue
        
        cleaned_transcript.append(text)
    
    return ' '.join(cleaned_transcript)

def split_transcript(transcript, max_tokens=2000):
    """Split the transcript into chunks that respect the token limit."""
    words = transcript.split()
    chunks = []
    chunk = []
    chunk_length = 0

    for word in words:
        chunk.append(word)
        chunk_length += len(word) + 1  # Approximate token length

        if chunk_length >= max_tokens:
            chunks.append(' '.join(chunk))
            chunk = []
            chunk_length = 0

    if chunk:
        chunks.append(' '.join(chunk))
    
    return chunks

def summarize_chunk(chunk):
    
    """Summarize a chunk of the transcript using GPT-3.5-turbo, formatted with markdown."""
    prompt = f"""Please summarize the following transcript by topic for a blog post in markdown format. 
    Use Heading 1 for main topics, Heading 2 for subtopics, and so on. Do not add any titles such as "Summary" or "Summary of Transcript".  
    Exclude any jokes or shenanigans:
    
    {chunk}
    
    Summary:"""
    
    
    
#     """Summarize a chunk of the transcript using GPT-3.5-turbo."""
#     prompt = f"""Please summarize the following transcript by topic for a blog post. Exclude any jokes or shenanigans:
    
#     {chunk}
    
#     Summary:"""
    
    try:
        response = openai.chat.completions.create(
        #response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes long transcripts by topic in markdown format."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=2000  # Limit response length to fit within the token limit
        )
        summary= response.choices[0].message.content
        #summary = response.choices[0].message['content']
    except openai.OpenAIError  as e:
        print(f"OpenAI API error: {e}")
        return None
    
    return summary

# def summarize_all_chunks(full_text):
#     """Summarize the entire concatenated text of all chunks."""
#     prompt = f"""Please provide a concise summary of the following text in markdown format, focusing on the main topics:
    
#     {full_text}
    
#     Summary:"""
    
#     try:
#         response = openai.chat.completions.create(
#             model="gpt-3.5-turbo",
#             messages=[
#                 {"role": "system", "content": "You are a helpful assistant that summarizes long texts in markdown format."},
#                 {"role": "user", "content": prompt}
#             ],
#             max_tokens=2000
#         )
#         overall_summary = response.choices[0].message.content
#         print(f"Overall Summarization finalized")
#     except openai.OpenAIError as e:
#         print(f"OpenAI API error: {e}")
#         return None
    
    
#     return overall_summary


def summarize_by_topic(transcript):
    """Summarize the entire transcript by breaking it into chunks."""
    chunks = split_transcript(transcript, max_tokens=2000)
    full_summary = ""

    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i+1} of {len(chunks)}")
        summary = summarize_chunk(chunk)
        if summary:
            full_summary += summary + "\n\n"
            
    # After all chunks are summarized and concatenated, summarize the concatenated text
    #overall_summary = summarize_all_chunks(full_summary)
    
    # Return the overall summary followed by the detailed chunk summaries
    #return [(overall_summary + "\n\n" + full_summary).strip(), chunks]

    return [full_summary.strip(),chunks]

def main(video_url):
    # Extract video ID from URL
    video_id = re.search(r'v=([a-zA-Z0-9_-]+)', video_url).group(1)
    
    # Get the transcript
    transcript = get_transcript(video_id)
    if transcript is None:
        return
    
    # Clean the transcript
    cleaned_transcript = clean_transcript(transcript)
    
    # Summarize by topic
    summary1 = summarize_by_topic(cleaned_transcript)
    chunks= summary1[1]
    summary=summary1[0]
    
    # Prompt for the output file name
    output_file_name = input("Enter the name of the file to save the summary (without extension): ").strip()
    output_file_name = output_file_name + ".md"
    
    # Output the final summary
    if summary:
        #print(summary)
        # Optionally, save the summary to a file
        with open(output_file_name, "w") as f:
            f.write(summary)
        print(f"Summary saved to {output_file_name}")
    else:
        print("Failed to generate summary")
    return summary1

# Example usage
video_url = "https://www.youtube.com/watch?v=xA5B6quoahY"  # Replace with your YouTube URL
#video_url = "https://www.youtube.com/watch?v=FM6kHcXpw98" # Replace with your YouTube URLhttps://www.youtube.com/watch?v=FM6kHcXpw98

main(video_url)


Processing chunk 1 of 53
Processing chunk 2 of 53
Processing chunk 3 of 53
Processing chunk 4 of 53
Processing chunk 5 of 53
Processing chunk 6 of 53
Processing chunk 7 of 53
Processing chunk 8 of 53
Processing chunk 9 of 53
Processing chunk 10 of 53
Processing chunk 11 of 53
Processing chunk 12 of 53
Processing chunk 13 of 53
Processing chunk 14 of 53
Processing chunk 15 of 53
Processing chunk 16 of 53
Processing chunk 17 of 53
Processing chunk 18 of 53
Processing chunk 19 of 53
Processing chunk 20 of 53
Processing chunk 21 of 53
Processing chunk 22 of 53
Processing chunk 23 of 53
Processing chunk 24 of 53
Processing chunk 25 of 53
Processing chunk 26 of 53
Processing chunk 27 of 53
Processing chunk 28 of 53
Processing chunk 29 of 53
Processing chunk 30 of 53
Processing chunk 31 of 53
Processing chunk 32 of 53
Processing chunk 33 of 53
Processing chunk 34 of 53
Processing chunk 35 of 53
Processing chunk 36 of 53
Processing chunk 37 of 53
Processing chunk 38 of 53
Processing chunk 39 o

['# Podcast Introduction\nThe all-in podcast features a conversation among four main speakers: J Cal (moderator), David Sachs (Rainman), David Friedberg (Sulan of Science), and Chamath Palihapitiya (Chairman Dictator). \n\n## Personal Updates and Comments\n### David Sachs\nSachs has lately been looking healthy and well-groomed. In answer to the question about his personal styling, he mentions that his barber comes to visit him.\n\n### David Friedberg\nFriedberg enjoys a life by the bay, loves the outdoors, and maintains a healthy diet. He recently celebrated a birthday with the moderator, Jason, which involved a long mountain bike ride and a family barbecue. He also mentions that he always brings his own food.\n\n### Jason\nJason has also just celebrated his 39th birthday.\n   \n### Chamath Palihapitiya\nChamath appears relaxed and casual in his attire. He has become a popular figure online, with his actions during the podcast making him the subject of internet memes. During the previo